In [ ]:
# PATH
if (!$env:PATH.Contains(".crc/bin/oc")) { $env:PATH = $env:HOME+"/.crc/bin/oc:"+$env:PATH }
# as kubeadmin
oc login -u kubeadmin https://api.crc.testing:6443

$ns = oc config view --minify -o 'jsonpath={..namespace}'
#"$($ns):egressips-reader"

In [ ]:
oc delete clusterrolebinding egressips-reader
oc delete serviceaccount egressips-reader
oc delete clusterrole egressips-reader

In [ ]:

@"
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1

metadata:
 name: egressips-reader

rules:
- apiGroups: ["network.openshift.io"]
  resources: ["netnamespaces"]
  verbs: ["get", "list"]
"@ | oc apply -f -

oc create serviceaccount egressips-reader
oc create clusterrolebinding egressips-reader --clusterrole egressips-reader --serviceaccount "$($ns):egressips-reader"

In [ ]:
$serviceAccount = "egressips-reader"
$secretName=oc get serviceaccount $serviceAccount -o json | ConvertFrom-Json `
| % { $_.secrets } `
| ? { $_.name.startsWith("$($serviceAccount)-token-") } `
| select -First 1 | % { $_.name }

# $secretName="egressips-reader-token-zc78k"
$secretName

# token from secretName
$token = oc get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
$token 

# ca.crt
$caCrtFilename = "./k8sca.crt"
oc get secret $secretName -o json  | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
$caCrtFilename 
# API server URL
$apiServerUrl = oc config view -o json | jq -c '.clusters[] | select(.name == ""api-crc-testing:6443"") | .cluster.server'

#$apiServerUrl = "https://api.crc.testing:6443"
$apiServerUrl

In [ ]:
/usr/bin/curl -v "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename 
#   | jq '.items[] | .metadata.name'

    



In [ ]:
"token: $token"

/usr/bin/curl -s "$apiServerUrl/apis/network.openshift.io/v1/netnamespaces" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename 